<a href="https://colab.research.google.com/github/fathursidiq/CNNtampalibrary/blob/main/SegmentaionsBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import glob
from tensorflow.keras.models import Sequential,load_model,save_model
from tensorflow.keras.layers import TimeDistributed, Conv1D, Activation, Bidirectional, LSTM, MaxPooling1D, Dropout, Flatten, Dense, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import hilbert
from scipy.signal import welch

In [ ]:
class DeCNNLSTM:
    def __init__(self, input_signal, target_signal):
        self.input_signal = input_signal
        self.target_signal = target_signal
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.model = Sequential()
        self.history = None
    def preprocess_signals(self):
        self.input_signal = self.scaler.fit_transform(self.input_signal.reshape(-1,1))
        self.input_signal = self.input_signal.reshape((-1, 1, 1))
        self.target_signal = self.target_signal.reshape((-1, 1, 1))

    def build_model(self, initial_learning_rate=0.01):
        signal_length = self.input_signal.shape[1]
        self.model.add(Conv1D(filters=64, kernel_size=10, input_shape=(signal_length, 1), padding='same'))
        self.model.add(Activation('relu'))
        pool_size = min(2, signal_length)
        self.model.add(MaxPooling1D(pool_size=pool_size))
        self.model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
        self.model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
        self.model.add(Dropout(0.2))
        self.model.add(TimeDistributed(Dense(units=64, activation='relu')))
        self.model.add(LSTM(units=64, return_sequences=True))
        self.model.add(Dense(1, activation='softmax', name='output'))
        optimizer = Adam(learning_rate=initial_learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy')
        self.model.summary()

    def train_model(self, epochs=10, batch_size=64, validation_data=None):
        if validation_data:
            validation_data = (validation_data[0].reshape((-1, 1, 1)), validation_data[1].reshape((-1, 1, 1)))
        def lr_scheduler(epoch, lr):
            if epoch % 10 == 0 and epoch != 0:
                return lr * 0.9
            else:
                return lr

        self.history = self.model.fit(
            self.input_signal, self.target_signal,
            epochs=epochs, batch_size=batch_size, verbose=1,
            validation_data=validation_data,
            callbacks=[LearningRateScheduler(lr_scheduler, verbose=1)]
        )

    def plot_loss(self):
        if self.history is not None:
            plt.plot(self.history.history['loss'], label='Training Loss')
            if 'val_loss' in self.history.history:
                plt.plot(self.history.history['val_loss'], label='Validation Loss')
            plt.title('Loss over Epochs')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            plt.show()

    def predict_new_signal(self, new_signal):
        new_signal = new_signal.reshape((-1, 1, 1))
        prediction = self.model.predict(new_signal)
        return prediction

    def save_model(self, model_path='/content/drive/MyDrive/thesis/delmodel/Segmentations_model_GLQ.h5'):
        self.model.save(model_path)
        print(f"Model saved to {model_path}")

    def load_model(self, model_path='/content/drive/MyDrive/thesis/delmodel/Segmentations_model_GLQ.h5'):
        self.model = load_model(model_path)
        print(f"Model loaded from {model_path}")


In [ ]:
def load_data_and_target_from_folder(data_folder_path, target_folder_path, delimiter=';'):
    data_files = glob.glob(os.path.join(data_folder_path, '*.CSV'))
    data_files.sort()
    data_list = []
    target_list = []
    target2_list = []
    for data_file in data_files:
        file_name = os.path.splitext(os.path.basename(data_file))[0]
        target_file = os.path.join(target_folder_path, f'{file_name}.CSV')
        if os.path.exists(target_file):
            df_data = pd.read_csv(data_file, delimiter=delimiter)[1000:9000]
            df_target = pd.read_csv(target_file, delimiter=delimiter)[1000:9000]
            data_list.append(df_data.filter(like='data').values)
            target_list.append(df_target.filter(like='pwave').values)
            target2_list.append(df_target.filter(like='swave').values)
    data_array = np.concatenate(data_list)
    target_array = np.concatenate(target_list)
    target2_array = np.concatenate(target2_list)
    return data_array, target_array, target2_array
data_folder_path = '/content/drive/MyDrive/thesis/filtered'
target_folder_path = '/content/drive/MyDrive/thesis/label'
data_array, target_array, target2_array = load_data_and_target_from_folder(data_folder_path, target_folder_path)

In [ ]:
das=data_array[100000:120000]
tar=target_array[100000:120000]
plt.plot(das*0.1, label='microtremor')
plt.plot(tar, label='p-wave')
plt.legend()
plt.show()

In [ ]:
class GaussLegendreQuadrature:
    def __init__(self, num_points=3):
        self.num_points = num_points
        self.points, self.weights = np.polynomial.legendre.leggauss(num_points)

    def integrate(self, function):
        result = np.sum(self.weights * function(self.points))
        return result

if __name__ == "__main__":
    def function_to_integrate(x):
        return x
    num_points = 4
    gauss_legendre = GaussLegendreQuadrature(num_points)
    x_vals = np.array(das)
    cumulative_results = []
    cumulative_sum = 0
    for x in x_vals:
        cumulative_sum += gauss_legendre.integrate(lambda x_val: function_to_integrate(x_val + x))
        cumulative_results.append(cumulative_sum)

    analytic_signal = hilbert(cumulative_results)
    hilbert_gauss_legendre = np.abs(analytic_signal)
    analytic_s = hilbert(x_vals)
    hilbert_envelope = np.abs(analytic_s)
    plt.figure(figsize=(12, 8))
    plt.subplot(4, 1, 1)
    plt.plot(x_vals, label='Original Sine Wave')
    plt.title('Original Sine Wave Function')
    plt.xlabel('x')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.subplot(4, 1, 2)
    plt.plot( np.array(cumulative_results), label='Cumulative Integration ', color='orange')
    plt.plot(tar)
    plt.title('Cumulative Integration of Sine Wave using Gaussian-Legendre Quadrature')
    plt.xlabel('x')
    plt.ylabel('Cumulative Amplitude')
    plt.legend()
    plt.subplot(4, 1, 3)
    plt.plot( hilbert_gauss_legendre, label='Hilbert_Gauss Transform Analysis', color='green')
    plt.title('Hilbert Transform Analysis')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.subplot(4, 1, 4)
    plt.plot( hilbert_envelope, label='Hilbert Transform Analysis', color='green')
    plt.title('Hilbert Transform Analysis')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
data_1=np.array(cumulative_results)#np.array(hilbert_gauss_legendre)
#data_2=target_array[:563301]
data_3=np.array(tar)
cnn_lstm_auto_filter = DeCNNLSTM(data_1, data_3)
cnn_lstm_auto_filter.preprocess_signals()
cnn_lstm_auto_filter.build_model()


In [ ]:
data_1=np.array(cumulative_results)#np.array(hilbert_gauss_legendre)
#data_2=target_array[:563301]
data_3=np.array(tar)
cnn_lstm_auto_filter = DeCNNLSTM(data_1, data_3)
cnn_lstm_auto_filter.preprocess_signals()
cnn_lstm_auto_filter.build_model()


In [ ]:
cnn_lstm_auto_filter.train_model(epochs=200, batch_size=512)
cnn_lstm_auto_filter.plot_loss()
cnn_lstm_auto_filter.save_model()

In [ ]:

data_1=np.array(hilbert_gauss_legendre)
#data_2=target_array[:563301]
data_3=np.array(tar)
model1 = DeCNNLSTM(data_1, data_3)
model1.load_model('/content/drive/MyDrive/thesis/delmodel/Segmentations_model_hilbert.h5')
new_signal = data_1[:20000]
predictions1 = model1.predict_new_signal(new_signal)
model2 = DeCNNLSTM(data_1, data_3)
model2.load_model('/content/drive/MyDrive/thesis/delmodel/Segmentations_model_GLQ.h5')
predictions2 = model2.predict_new_signal(new_signal)
print(predictions2)
print(predictions1)
threshold = 3.8940729e-05
binary_predictions1 = (predictions1 > threshold).astype(int)
binary_predictions2 = (predictions2 > threshold).astype(int)
plt.figure(figsize=(15, 5))
plt.plot(das[:20000]*10, label='Original Signal')
plt.plot(predictions1.flatten(), label='s-wave', alpha=0.5)
plt.plot(predictions2.flatten(), label='p-wave', alpha=0.5)
plt.legend()
plt.show()


filters untuk denoising

perintah untuk menjalankan

untuk training tiap file dalam 1 folder

In [ ]:


model1 = DeCNNLSTM(data_1, data_3)
model1.load_model('/content/drive/MyDrive/thesis/delmodel/Segmentations_model_s-wave.h5')
new_signal = data_1[:7000]
predictions1 = model1.predict_new_signal(new_signal)

model2 = DeCNNLSTM(data_1, data_3)
model2.load_model('/content/drive/MyDrive/thesis/delmodel/Segmentations_model.h5')
predictions2 = model2.predict_new_signal(new_signal)
threshold = 3.8940729e-05
binary_predictions1 = (predictions1 > threshold).astype(int)
binary_predictions2 = (predictions2 > threshold).astype(int)
plt.figure(figsize=(15, 5))
plt.plot(das[:7000]*50, label='Original Signal')
#plt.plot(binary_predictions1.flatten(), label='microtremor', alpha=0.7)
plt.plot(binary_predictions2.flatten(), label='p-wave', alpha=0.7)
plt.legend()
plt.show()
